In [2]:
# ON UBUNTU
import sys 
sys.path.append('../')
sys.path.append('../../')

from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy import units as u
from astroquery.skyview import SkyView

import pandas as pd 
import pickle
import subprocess
import numpy as np
import matplotlib.pyplot as plt

from astropy.visualization import ZScaleInterval
from astropy.table import Table,Column
from astropy.stats import sigma_clipped_stats, SigmaClip
from tractor import *
from random import gauss
from photutils import Background2D, SExtractorBackground
from astropy.visualization import ZScaleInterval

#from ipywidgets import interact, interactive, fixed, interact_manual
#import ipywidgets as widgets

# My Modules
import RetrieveSource as RS
import RetrieveSourceMRD as RSMRD
import TractorTools as Trac
import TractorToolsMRD as TracMRD
import Heasarc as Hea
import HeasarcMRD as HeaMRD
import EstimateBackground as EB
from PSF import *

from astropy.wcs import WCS
from astropy.stats import sigma_clipped_stats, SigmaClip
from photutils import make_source_mask

import glob
import time

In [3]:
# Set the name of the source you want:

source_name='smc-obs40454-4048'
field='40454'
base="/Volumes/MRD_Backup/SwiftData/Astrometry/SMC/"
csvfile=base+'SMC_combined_1_1.csv'

all_data = pd.read_csv(csvfile)
source_data = all_data[all_data['name'] == source_name]

cats = glob.glob(base+'*um2*'+field+'_1_1.dat')
files = glob.glob(base+'*um2*'+field+'_1_1.new')

uvm2_x = np.int(source_data['uvm2_pix_x'])
uvm2_y = np.int(source_data['uvm2_pix_y'])

xdim = [uvm2_x-50,uvm2_x+50]
ydim = [uvm2_y-50,uvm2_y+50]


# I have added code so that it will run a source through tractor if it is some threshold above the background
# So as a result I've lowered the 'cutoff' here.
# "../../Optical/Zaritsky/table1_smc.dat"
Sources = RSMRD.get_meta().with_hdu(fits.open(files[0])[0],cutoff=20.5,xdim=xdim,ydim=ydim,
                                 fits_origin=0,optical_catalog = cats[0],
                                   astrometry=True)

# Correct data and guesses for exposure time. Could decide to do that internally to RS.
exp_time = Sources.header["EXPOSURE"]
Sources.data = Sources.data/exp_time
Sources.source_intensities = Sources.source_intensities/exp_time

## Background
Background = EB.BkgdEstimator(Sources,n_pix = [20,20])

#Grab theoretical.
pix_scale = np.abs(Sources.cdelt)*3600.
psf_object = psf_fit(pixel_per_arsecond = 1/pix_scale,
                     uvfilter = "UVM2", 
                     width = 23, 
                     cog_file="../../Photometry/swureef20041120v104.fits").psf

t1 = time.time()
TractorObject1 = TracMRD.PhotometryTools(Sources,Background,
                        psf_filename=psf_object,
                        fits_table_sname="Trac_SMC_test1.fits",
                        threshold=1.5,
                        fit_positions = False)


TractorObject2 = TracMRD.PhotometryTools(Sources,Background,
                        psf_filename=psf_object,
                        fits_table_sname="Trac_SMC_test2.fits",
                        threshold=1.5,
                        fit_positions = True)

print('Tractor Time',(time.time()-t1)/60)


hea1 = HeaMRD.HeasarcRoutines("Trac_SMC_test1.fits", "UVM2")
hea2 = HeaMRD.HeasarcRoutines("Trac_SMC_test2.fits", "UVM2")



# Plot outputs from this:
#x,y = Sources.pixel_positions
x = TractorObject1.x
y = TractorObject1.y
f,axes = plt.subplots(1,3,figsize=(30,20))

interval1 = ZScaleInterval().get_limits(TractorObject1.image)
interval2 = ZScaleInterval().get_limits(TractorObject1.model)
interval3 = ZScaleInterval().get_limits(TractorObject1.image - TractorObject1.model)

axes[0].imshow(TractorObject1.image,origin="lower",vmin=interval1[0],vmax=interval1[1])
axes[0].scatter(x,y,s=5,c='red')
axes[1].imshow(TractorObject1.model,origin="lower",vmin=interval1[0],vmax=interval1[1])
axes[1].scatter(x,y,s=5,c='red')
axes[2].imshow(TractorObject1.image - TractorObject1.model,origin="lower",vmin=interval3[0],vmax=interval3[1])
axes[2].scatter(x,y,s=5,c='red')
plt.show()

x = TractorObject2.x
y = TractorObject2.y
f,axes = plt.subplots(1,3,figsize=(30,20))

interval1 = ZScaleInterval().get_limits(TractorObject2.image)
interval2 = ZScaleInterval().get_limits(TractorObject2.model)
interval3 = ZScaleInterval().get_limits(TractorObject2.image - TractorObject2.model)

axes[0].imshow(TractorObject2.image,origin="lower",vmin=interval1[0],vmax=interval1[1])
axes[0].scatter(x,y,s=5,c='red')
axes[1].imshow(TractorObject2.model,origin="lower",vmin=interval1[0],vmax=interval1[1])
axes[1].scatter(x,y,s=5,c='red')
axes[2].imshow(TractorObject2.image - TractorObject2.model,origin="lower",vmin=interval3[0],vmax=interval3[1])
axes[2].scatter(x,y,s=5,c='red')
plt.show()

##Print the outputs of the photometry for sources close the the pixels above. 
xdim2 = [uvm2_x-5-xdim[0],uvm2_x+5-xdim[0]]
ydim2 = [uvm2_y-5-ydim[0],uvm2_y+5-ydim[0]]


v1 = pd.read_csv('Trac_SMC_test1.csv')
v2 = pd.read_csv('Trac_SMC_test2.csv')

v1_sub = v1[(v1['PIX_X'] > xdim2[0]) & (v1['PIX_X'] < xdim2[1]) & (v1['PIX_Y'] > ydim2[0]) & (v1['PIX_Y'] < ydim2[1])]
print(v1_sub)

v2_sub = v2[(v2['PIX_X'] > xdim2[0]) & (v2['PIX_X'] < xdim2[1]) & (v2['PIX_Y'] > ydim2[0]) & (v2['PIX_Y'] < ydim2[1])]
print(v2_sub)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55890.873634 from DATE-OBS.
Set MJD-END to 55890.877627 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55890.873634 from DATE-OBS.
Set MJD-END to 55890.877627 from DATE-END'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]
Set MJD-OBS to 55890.873634 from DATE-OBS.
Set MJD-END to 55890.877627 from DATE-END'. [astropy.wcs.wcs]


INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]
Masked Sigma Clipped

Scaling is 1.0044330824124 ArcSeconds per Pixel
Image needs to be 23 pixels to be 23 ArcSeconds in size.
Sum Before Normalization  616.9218729335043
Sum After Normalization  0.9999999999999998


TypeError: Cannot cast array data from dtype('float64') to dtype('float32') according to the rule 'safe'